# Instagram Agent Demo

This notebook demonstrates how to use the `ig_agent` library to generate Instagram carousel posts with AI.

## 1. Setup and Installation

First, ensure you have all the necessary dependencies installed:

In [1]:
# Install dependencies if needed
!pip install -q langchain_xai cairosvg
!pip install -q -e .


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


## 2. Basic Usage - Direct API

The simplest way to use the Instagram Agent is through its direct API, which provides methods for each step of the content generation and posting process.

In [2]:
import os
import logging
from pathlib import Path
from IPython.display import Image, display, HTML
from langchain_xai import ChatXAI
from ig_agent import InstagramAgent
import os
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("ig_agent_demo")
EPOCH = 2

In [3]:
# Create base output directory 
# Now outputs will be stored in datetime subdirectories
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Initialize language model
# You can use any LangChain compatible LLM here
llm = ChatXAI(model="grok-3-mini-beta")

# Initialize the Instagram agent
# Output will automatically be stored in a datetime-based directory structure
# For example: output/2025-04-20/20250420_154900/
agent = InstagramAgent(
    llm=llm,
    output_dir=output_dir,
    # Optional: provide credentials for notification and posting
    email_user=os.environ.get("EMAIL_USER"),
    email_password=os.environ.get("EMAIL_PASSWORD"),
    instagram_username=os.environ.get("INSTAGRAM_USERNAME"),
    instagram_password=os.environ.get("INSTAGRAM_PASSWORD")
)

### 2.1 Generate Content

First, we generate the content for our Instagram carousel post based on a request:

In [4]:
from datetime import datetime
today = datetime.now().strftime('%Y-%b-%d')

In [5]:
import pandas as pd
for epoch in range(EPOCH):
    print(f'Starting Epoch {epoch+1}')
    df = pd.read_json('data/content_history.json')
    latest_10rows_df =df.iloc[-30:]
    # Generate content
    request = f"""
    今日係 {today}, 星期{datetime.now().weekday()}.
    最近10近貼文：{
        latest_10rows_df.to_dict()
    }
    不要重覆內容。
    標題要eyes catching，非常搶眼，引人入勝貼留下懸念去睇內容頁。
    選取以下一內容製作今日的貼文，你可選擇是否再上網搜尋資料，特別係新聞話題，必須上網。不要用中國字，用香港字、繁體中文撰寫。
    | **內容類型**         | **具體題材**                                                                 | **預期效果**                                                                 |
    |----------------------|------------------------------------------------------------------------------|------------------------------------------------------------------------------|
    | **教育性貼文**       | - 解釋AI基本概念（如機器學習、深度學習、自然語言處理）<br>- 數據科學工具介紹（如Python、R、Tableau）<br>- 簡單算法原理（e.g., 線性回歸、決策樹） | 吸引初學者，提供入門知識，增加帳號專業性。                                   |
    | **實用技巧分享**     | - Python/R程式碼片段（e.g., 數據清理、視覺化）<br>- Jupyter Notebook使用技巧<br>- 免費學習資源推薦（課程、書籍、YouTube頻道） | 提供實用價值，吸引技術愛好者，提升互動率。                                   |
    | **案例分析**         | - AI應用案例（e.g., 醫療影像診斷、推薦系統）<br>- 數據科學項目實例（e.g., 銷售預測）<br>- 失敗案例分析（e.g., 模型過擬合問題） | 展示AI/數據科學的實際應用，激發靈感，吸引專業人士。                           |
    | **趨勢與新聞**       | - 最新AI技術進展（e.g., 大語言模型、生成式AI）<br>- 數據科學行業趨勢（e.g., 自動化數據分析）<br>- 倫理問題討論（e.g., AI偏見） | 保持內容新鮮，吸引關注行業動態的受眾。                                       |
    | **互動內容**         | - 問答貼文（e.g., “你用哪個數據科學工具？”）<br>- 投票（e.g., “Python vs R，哪個更好？”）<br>- 挑戰（e.g., 7天學會Pandas） | 提高粉絲參與度，建立社群感。                                               |
    | **幕後花絮**         | - 數據科學家/工程師一天的工作分享<br>- 項目開發過程（e.g., 從數據收集到模型部署）<br>- 學習路徑分享（e.g., 自學數據科學的經驗） | 增加人性化元素，拉近與受眾的距離。                                           |
    | **迷因與輕鬆內容**   | - AI/數據科學相關迷因（e.g., “當模型訓練了10小時還是沒收斂”）<br>- 程式員幽默（e.g., debug的痛苦）<br>- 數據科學家日常趣事 | 增加趣味性，吸引年輕受眾，提高轉發率。                                       |
    | **面試內容**   | - AI/數據科學相關職位的面試內容，以FAANG公司為主。 | 增加趣味性，吸引年輕受眾，提高轉發率。                                       |

    """
    content_json = agent.generate_content(request)
    print('Contents are ready.')
    # Generate images
    images = agent.generate_images(content_json)
    print('Images generation are ready.')
    # Try with explicit error handling
    instagram_status = agent.post_to_instagram()
    print(f"Posted to Instagram: {instagram_status['posted']}")
    print(f'Epoch: {epoch+1} now.')

2025-04-30 10:14:53,139 - ig_agent - INFO - Generating content for request: 
    今日係 2025-Apr-30, 星期2.
    最近10近貼文：{'date': {12: Timestamp('2025-04-23 13:21:04.506031'), 13: Timestamp('2025-04-23 13:21:46.053301'), 14: Timestamp('2025-04-23 13:26:41.557815'), 15: Timestamp('2025-04-23 13:37:50.518470'), 16: Timestamp('2025-04-23 13:42:42.135440'), 17: Timestamp('2025-04-24 11:46:34.359186'), 18: Timestamp('2025-04-24 12:17:12.846487'), 19: Timestamp('2025-04-25 13:40:44.213580'), 20: Timestamp('2025-04-27 19:39:27.693493'), 21: Timestamp('2025-04-27 20:06:39.550101'), 22: Timestamp('2025-04-27 20:10:24.162216'), 23: Timestamp('2025-04-27 20:12:24.561066'), 24: Timestamp('2025-04-27 20:13:26.347846'), 25: Timestamp('2025-04-27 20:14:25.523182'), 26: Timestamp('2025-04-27 20:15:34.027884'), 27: Timestamp('2025-04-27 20:16:33.632171'), 28: Timestamp('2025-04-27 20:20:38.622609'), 29: Timestamp('2025-04-27 20:21:39.580129'), 30: Timestamp('2025-04-27 20:22:37.434548'), 31: Timestamp('2025-

Starting Epoch 1


2025-04-30 10:15:07,058 - httpx - INFO - HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-30 10:15:08,177 - primp - INFO - response: https://html.duckduckgo.com/html 200
2025-04-30 10:15:36,240 - httpx - INFO - HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-30 10:16:16,512 - httpx - INFO - HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-30 10:16:16,531 - ig_agent.content_agent - INFO - Generated unique content on attempt 1
2025-04-30 10:16:16,532 - ig_agent - INFO - Generating images from content


Contents are ready.


2025-04-30 10:16:16,759 - ig_agent.template_processor - WARNING - PyCairo is not available, falling back to other methods
2025-04-30 10:16:16,759 - ig_agent.template_processor - INFO - CairoSVG version: 2.7.1
2025-04-30 10:16:16,761 - ig_agent.template_processor - INFO - Attempting SVG conversion with rsvg-convert for cover...
2025-04-30 10:16:17,061 - ig_agent.template_processor - INFO - Successfully converted SVG to PNG using rsvg-convert
2025-04-30 10:16:17,062 - ig_agent.template_processor - INFO - Generated cover image: output/2025-04-30/20250430_101452/cover.png
2025-04-30 10:16:17,064 - ig_agent.template_processor - INFO - Attempting SVG conversion with rsvg-convert for content page 1...
2025-04-30 10:16:17,196 - ig_agent.template_processor - INFO - Successfully converted SVG to PNG using rsvg-convert
2025-04-30 10:16:17,197 - ig_agent.template_processor - INFO - Generated content image 1: output/2025-04-30/20250430_101452/content_01.png
2025-04-30 10:16:17,198 - ig_agent.templa

...

Contents are ready.


2025-04-30 10:22:52,927 - ig_agent.template_processor - INFO - Successfully converted SVG to PNG using rsvg-convert
2025-04-30 10:22:52,928 - ig_agent.template_processor - INFO - Generated cover image: output/2025-04-30/20250430_101452/cover.png
2025-04-30 10:22:52,929 - ig_agent.template_processor - INFO - Attempting SVG conversion with rsvg-convert for content page 1...
2025-04-30 10:22:53,063 - ig_agent.template_processor - INFO - Successfully converted SVG to PNG using rsvg-convert
2025-04-30 10:22:53,063 - ig_agent.template_processor - INFO - Generated content image 1: output/2025-04-30/20250430_101452/content_01.png
2025-04-30 10:22:53,064 - ig_agent.template_processor - INFO - Attempting SVG conversion with rsvg-convert for content page 2...
2025-04-30 10:22:53,197 - ig_agent.template_processor - INFO - Successfully converted SVG to PNG using rsvg-convert
2025-04-30 10:22:53,197 - ig_agent.template_processor - INFO - Generated content image 2: output/2025-04-30/20250430_101452/c

Images generation are ready.


2025-04-30 10:22:54,245 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/api/v1/launcher/sync/ (269.0.0.18.75, OnePlus 6T Dev)
2025-04-30 10:22:54,543 - instagrapi - INFO - https://i.instagram.com/api/v1/accounts/login/
2025-04-30 10:22:58,139 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/api/v1/accounts/login/ (269.0.0.18.75, OnePlus 6T Dev)
2025-04-30 10:22:59,146 - instagrapi - INFO - https://i.instagram.com/api/v1/feed/reels_tray/
2025-04-30 10:22:59,521 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/api/v1/feed/reels_tray/ (269.0.0.18.75, OnePlus 6T Dev)
2025-04-30 10:23:00,527 - instagrapi - INFO - https://i.instagram.com/api/v1/feed/timeline/
2025-04-30 10:23:02,424 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/api/v1/feed/timeline/ (269.0.0.18.75, OnePlus 6T Dev)
2025-04-30 10:23:04,337 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/ru

Posted to Instagram: True
Epoch: 2 now.
